In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import matplotlib.cm as cm
import matplotlib.colors as colors

!pip install folium
import folium 

from sklearn.cluster import KMeans

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
wiki = requests.get(url)
soup = BeautifulSoup(wiki.text, "html.parser")

In [3]:
columns=["Postalcode","Borough","Neighbourhood"]
soup = BeautifulSoup(wiki.text, "html.parser")

table = soup.find("table")
table_rows = table.find_all("tr")

rowdata = []
for tr in table_rows:
    td = tr.find_all("td")
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        rowdata.append(row)

df = pd.DataFrame(rowdata, columns = ["PostalCode", "Borough", "Neighborhood"])
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [4]:
df = df[df.Borough!='Not assigned']
df.reset_index(drop = True, inplace = True)

df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [5]:
#Read CSV
geo = pd.read_csv("http://cocl.us/Geospatial_data")
geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [6]:
geo.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
#newdf = df.merge(geo, left_on = "Postalcode")
#newdf.head()

newdf = df.merge(geo, on = "PostalCode")
newdf.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


### Use geopy library to get the latitude and longitude values of Toronto

In [7]:
address = 'Toronto, Ca'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [8]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(newdf['Latitude'], newdf['Longitude'], newdf['Borough'], newdf['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Define Foursquare Credentials and Version

In [9]:
CLIENT_ID = 'KSGDR4FWKJO0YC3GBVAAQ5UZCGKAQ1VYWW20GWWLF2OMPBOQ' # your Foursquare ID
CLIENT_SECRET = 'XHUPLNUPCQW15GGCJKVDKDHD1K1ICWY521NVFB4TBSSECNA5' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KSGDR4FWKJO0YC3GBVAAQ5UZCGKAQ1VYWW20GWWLF2OMPBOQ
CLIENT_SECRET:XHUPLNUPCQW15GGCJKVDKDHD1K1ICWY521NVFB4TBSSECNA5


## Let's explore the first neighborhood in our dataframe

In [10]:
newdf.loc[0, 'Neighborhood']

'Parkwoods'

In [11]:
neighborhood_latitude = newdf.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = newdf.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = newdf.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, neighborhood_latitude, neighborhood_longitude))

Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.


#### Now write the code to run the above function on each neighborhood and create a new dataframe called manhattan_venues

# Explore Neighborhoods in Toronto

In [12]:
LIMIT = 50
radius = 500
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [14]:
toronto_venues = getNearbyVenues(names = newdf['Neighborhood'],
                                   latitudes = newdf['Latitude'],
                                   longitudes = newdf['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

In [15]:
#check the size of the resulting dataframe
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,649 Variety,43.754513,-79.331942,Convenience Store
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


# Analyze Each Neighborhood


In [16]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
#examine the new dataframe size
toronto_onehot.shape

(1694, 253)

In [18]:
# group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Agincourt,0.000000,0.000,0.000000,0.0000,0.000,0.000,0.000,0.000000,0.0,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.0,0.0,0.00
1,"Alderwood, Long Branch",0.000000,0.000,0.000000,0.0000,0.000,0.000,0.000,0.000000,0.0,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.0,0.0,0.00
2,"Bathurst Manor, Wilson Heights, Downsview North",0.000000,0.000,0.000000,0.0000,0.000,0.000,0.000,0.000000,0.0,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.0,0.0,0.00
3,Bayview Village,0.000000,0.000,0.000000,0.0000,0.000,0.000,0.000,0.000000,0.0,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.0,0.0,0.00
4,"Bedford Park, Lawrence Manor East",0.000000,0.000,0.000000,0.0000,0.000,0.000,0.000,0.037037,0.0,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.0,0.0,0.00
5,Berczy Park,0.000000,0.000,0.000000,0.0000,0.000,0.000,0.000,0.000000,0.0,...,0.00,0.020000,0.000000,0.000000,0.000000,0.000000,0.000,0.0,0.0,0.00
6,"Birch Cliff, Cliffside West",0.000000,0.000,0.000000,0.0000,0.000,0.000,0.000,0.000000,0.0,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.0,0.0,0.00
7,"Brockton, Parkdale Village, Exhibition Place",0.080000,0.000,0.000000,0.0000,0.000,0.000,0.000,0.000000,0.0,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.0,0.0,0.00
8,"Business reply mail Processing Centre, South C...",0.052632,0.000,0.000000,0.0000,0.000,0.000,0.000,0.000000,0.0,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.0,0.0,0.00
9,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.000,0.062500,0.0625,0.125,0.125,0.125,0.000000,0.0,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.0,0.0,0.00


In [19]:
#confirm the new size
toronto_grouped.shape

(95, 253)

In [20]:
#print each neighborhood along with the top 5 most common venues
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0                     Lounge  0.25
1  Latin American Restaurant  0.25
2             Breakfast Spot  0.25
3               Skating Rink  0.25
4                Yoga Studio  0.00


----Alderwood, Long Branch----
            venue  freq
0     Pizza Place  0.25
1        Pharmacy  0.12
2     Coffee Shop  0.12
3  Sandwich Place  0.12
4             Pub  0.12


----Bathurst Manor, Wilson Heights, Downsview North----
               venue  freq
0        Coffee Shop  0.09
1               Bank  0.09
2  Mobile Phone Shop  0.04
3         Restaurant  0.04
4   Sushi Restaurant  0.04


----Bayview Village----
                 venue  freq
0  Japanese Restaurant  0.25
1                 Bank  0.25
2   Chinese Restaurant  0.25
3                 Café  0.25
4          Yoga Studio  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0  Italian Restaurant  0.11
1      Sandwich Place  0.07
2         Coffee Shop  0.07
3          Restaura

                           venue  freq
0  Vegetarian / Vegan Restaurant  0.08
1                           Café  0.08
2             Mexican Restaurant  0.06
3                    Coffee Shop  0.06
4                   Dessert Shop  0.04


----Kingsview Village, St. Phillips, Martin Grove Gardens, Richview Gardens----
               venue  freq
0  Mobile Phone Shop  0.33
1               Park  0.33
2     Sandwich Place  0.33
3        Yoga Studio  0.00
4             Lounge  0.00


----Lawrence Manor, Lawrence Heights----
                    venue  freq
0  Furniture / Home Store   0.2
1          Clothing Store   0.2
2   Vietnamese Restaurant   0.1
3             Event Space   0.1
4             Coffee Shop   0.1


----Lawrence Park----
                venue  freq
0  Dim Sum Restaurant  0.25
1                Park  0.25
2         Swim School  0.25
3            Bus Line  0.25
4   Mobile Phone Shop  0.00


----Leaside----
                    venue  freq
0     Sporting Goods Shop  0.09
1            

               venue  freq
0  Indian Restaurant  0.10
1     Sandwich Place  0.10
2        Yoga Studio  0.05
3           Pharmacy  0.05
4       Burger Joint  0.05


----Toronto Dominion Centre, Design Exchange----
                venue  freq
0         Coffee Shop  0.16
1                Café  0.14
2  Seafood Restaurant  0.06
3          Restaurant  0.06
4            Beer Bar  0.04


----University of Toronto, Harbord----
                 venue  freq
0                 Café  0.17
1               Bakery  0.06
2           Restaurant  0.06
3                  Bar  0.06
4  Japanese Restaurant  0.06


----Victoria Village----
                        venue  freq
0                 Pizza Place  0.17
1                 Coffee Shop  0.17
2                Hockey Arena  0.17
3       Portuguese Restaurant  0.17
4  Financial or Legal Service  0.17


----Westmount----
            venue  freq
0     Pizza Place  0.29
1  Sandwich Place  0.14
2    Intersection  0.14
3  Discount Store  0.14
4     Coffee Shop  0.

### Let's put that into a pandas dataframe

In [21]:
#write a function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [22]:
#create the new dataframe and display the top 10 venues for each neighborhood

num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Latin American Restaurant,Lounge,Skating Rink,Breakfast Spot,College Rec Center,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant
1,"Alderwood, Long Branch",Pizza Place,Pub,Coffee Shop,Pharmacy,Gym,Sandwich Place,Pool,Women's Store,Deli / Bodega,Department Store
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Shopping Mall,Pharmacy,Restaurant,Supermarket,Sushi Restaurant,Deli / Bodega,Middle Eastern Restaurant,Convenience Store
3,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
4,"Bedford Park, Lawrence Manor East",Italian Restaurant,Sandwich Place,Restaurant,Coffee Shop,Thai Restaurant,Liquor Store,Indian Restaurant,Fast Food Restaurant,Butcher,Café


# Cluster Neighborhoods

In [23]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([3, 1, 3, 3, 3, 3, 3, 3, 3, 3], dtype=int32)

In [24]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Label', kmeans.labels_)

toronto_merged = newdf

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,0.0,Convenience Store,Park,Food & Drink Shop,Women's Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
1,M4A,North York,Victoria Village,43.725882,-79.315572,3.0,Portuguese Restaurant,Pizza Place,Coffee Shop,Hockey Arena,French Restaurant,Financial or Legal Service,Diner,Department Store,Dessert Shop,Dim Sum Restaurant
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,3.0,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Café,Theater,Spa,Shoe Store,Restaurant
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,3.0,Furniture / Home Store,Clothing Store,Gift Shop,Coffee Shop,Boutique,Event Space,Vietnamese Restaurant,Accessories Store,Concert Hall,Eastern European Restaurant
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,3.0,Coffee Shop,Diner,Yoga Studio,Bar,Beer Bar,Smoothie Shop,Sandwich Place,Burrito Place,Café,Portuguese Restaurant


In [ ]:
# create map
map_clusters = folium.Map(location = [latitude, longitude], zoom_start = 11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

#### Cluster1

In [30]:
toronto_merged.loc[toronto_merged['Cluster Label'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,0.0,Convenience Store,Park,Food & Drink Shop,Women's Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
21,York,0.0,Park,Women's Store,Pool,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant
52,North York,0.0,Park,Women's Store,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
64,York,0.0,Park,Women's Store,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
66,North York,0.0,Park,Convenience Store,Women's Store,Deli / Bodega,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run
85,Scarborough,0.0,Park,Playground,Women's Store,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant
91,Downtown Toronto,0.0,Park,Playground,Trail,Discount Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Distribution Center


#### Cluster2

In [31]:
toronto_merged.loc[toronto_merged['Cluster Label'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,East York,1.0,Pizza Place,Bank,Pharmacy,Gastropub,Gym / Fitness Center,Athletics & Sports,Café,Intersection,Women's Store,Discount Store
50,North York,1.0,Pizza Place,Gym,Women's Store,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant
63,York,1.0,Brewery,Pizza Place,Grocery Store,Convenience Store,Women's Store,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
70,Etobicoke,1.0,Pizza Place,Chinese Restaurant,Coffee Shop,Sandwich Place,Discount Store,Intersection,Distribution Center,Dessert Shop,Dim Sum Restaurant,Diner
72,North York,1.0,Pharmacy,Pizza Place,Coffee Shop,Bank,Distribution Center,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant
82,Scarborough,1.0,Pizza Place,Convenience Store,Fast Food Restaurant,Thai Restaurant,Fried Chicken Joint,Italian Restaurant,Chinese Restaurant,Bank,Gas Station,Noodle House
89,Etobicoke,1.0,Pizza Place,Grocery Store,Fried Chicken Joint,Pharmacy,Video Store,Beer Store,Sandwich Place,Fast Food Restaurant,Discount Store,Women's Store
93,Etobicoke,1.0,Pizza Place,Pub,Coffee Shop,Pharmacy,Gym,Sandwich Place,Pool,Women's Store,Deli / Bodega,Department Store


#### Cluster3

In [32]:
toronto_merged.loc[toronto_merged['Cluster Label'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Scarborough,2.0,Fast Food Restaurant,Women's Store,Event Space,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run


#### Cluster4

In [33]:
toronto_merged.loc[toronto_merged['Cluster Label'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,3.0,Portuguese Restaurant,Pizza Place,Coffee Shop,Hockey Arena,French Restaurant,Financial or Legal Service,Diner,Department Store,Dessert Shop,Dim Sum Restaurant
2,Downtown Toronto,3.0,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Café,Theater,Spa,Shoe Store,Restaurant
3,North York,3.0,Furniture / Home Store,Clothing Store,Gift Shop,Coffee Shop,Boutique,Event Space,Vietnamese Restaurant,Accessories Store,Concert Hall,Eastern European Restaurant
4,Downtown Toronto,3.0,Coffee Shop,Diner,Yoga Studio,Bar,Beer Bar,Smoothie Shop,Sandwich Place,Burrito Place,Café,Portuguese Restaurant
7,North York,3.0,Gym,Coffee Shop,Beer Store,Japanese Restaurant,Restaurant,Café,Dim Sum Restaurant,Caribbean Restaurant,Supermarket,Discount Store
9,Downtown Toronto,3.0,Coffee Shop,Café,Theater,Clothing Store,Bookstore,Italian Restaurant,Ramen Restaurant,Burrito Place,Electronics Store,Shopping Mall
10,North York,3.0,Park,Japanese Restaurant,Sushi Restaurant,Pub,Discount Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Distribution Center
12,Scarborough,3.0,Bar,Women's Store,Falafel Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant
13,North York,3.0,Gym,Coffee Shop,Beer Store,Japanese Restaurant,Restaurant,Café,Dim Sum Restaurant,Caribbean Restaurant,Supermarket,Discount Store
14,East York,3.0,Dance Studio,Park,Skating Rink,Beer Store,Video Store,Athletics & Sports,Curling Ice,Bus Stop,Discount Store,Dessert Shop


#### Cluster5

In [34]:
toronto_merged.loc[toronto_merged['Cluster Label'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
57,North York,4.0,Baseball Field,Food Service,Women's Store,Event Space,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant
101,Etobicoke,4.0,Baseball Field,Women's Store,Falafel Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant
